# BoxMojo Dataset

In [1]:
# Data wrangling
import pandas as pd
import numpy as np
import re

# Time
import time
import datetime
from datetime import datetime as dt
import dateutil.parser

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# URL Grabbing
import requests

# Scraping / Searching
from bs4 import BeautifulSoup
import re

# Misc
import pickle
from pprint import pprint

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.2f' % x) # reduces the amount of decimals when printing the df's

# Inputting the Data

I was having issues with the way the data was formatted using scrapy, so I just readline by line, then evaluate the JSON since it works as a python dictionary. 

In [2]:
filename = 'test_run.jl'
filename_r = './bomojo_scrap/' + filename

with open(filename_r, 'r') as fh:
    fstr = fh.readlines()

In [3]:
# this was a test string to find an issue with the eval statement, use this if you can figure it out! :)
# test_str = '{"The Boxtrolls": {"domestic_gross_adj": "$52,992,000", "foreign_unadj": "\u00a0$58,447,728", "worldwide_gross_unadj": "$109,285,033", "release_date": "September 26, 2014", "genre": "Animation", "runtime": "1 hrs. 37 min.", "mpaa_rating": "PG", "production_budget_adj": "$60 million", "opening_weekend_adj": "\u00a0$18,493,900", "number_of_theaters": "\u00a03,464 theaters", "close_date": "\u00a0February 5, 2015", "days_in_theater": "\u00a0133 days / 19 weeks", "directors": ["Graham Annable", "Anthony Stacchi"], "writers": ["Irena Brignull", "Adam Pava"], "actors": ["Ben Kingsley", "Toni Collette", "Elle Fanning", "Jared Harris", "Simon Pegg", "Nick Frost", "Richard Ayoade ", "Tracy Morgan"], "producers": ["Travis Knight"], "composers": ["Dario Marianelli"]}}'

In [4]:
# grabs all the movies
movies = fstr

# replaces all 'null' strings with "null" because when evaluating the expression, it can't use the keyword null
movies = [x.replace('null','"null"') for x in movies]

# evaluates the expression to be a list of dicts!
# movies_eval = [eval(x) for x in movies]
movies_eval = []

print('Lines causing eval errors:\n')
for movie_json in movies:
    try:
        movies_eval.append(eval(movie_json))
    except:
        print(movie_json)

Lines causing eval errors:

{"The Boxtrolls": {"domestic_gross_adj": "$52,992,000", "foreign_unadj": "\u00a0$58,447,728", "worldwide_gross_unadj": "$109,285,033", "release_date": "September 26, 2014", "genre": "Animation", "runtime": "1 hrs. 37 min.", "mpaa_rating": "PG", "production_budget_adj": "$60 million", "opening_weekend_adj": "\u00a0$18,493,900", "number_of_theaters": "\u00a03,464 theaters", "close_date": "\u00a0February 5, 2015", "days_in_theater": "\u00a0133 days / 19 weeks", "directors": ["Graham Annable", "Anthony Stacchi"], "writers": ["Irena Brig"null"", "Adam Pava"], "actors": ["Ben Kingsley", "Toni Collette", "Elle Fanning", "Jared Harris", "Simon Pegg", "Nick Frost", "Richard Ayoade ", "Tracy Morgan"], "producers": ["Travis Knight"], "composers": ["Dario Marianelli"]}}



In [5]:
# creates a DataFrame for each of the dictionaries in the eval and concats them all
df = pd.concat([pd.DataFrame(x) for x in movies_eval], axis=1)

In [6]:
# transposes the columns and the index
df = df.T

# Checking the data

In [7]:
# check columns for any problems
df.columns

Index(['actors', 'close_date', 'composers', 'days_in_theater', 'directors',
       'domestic_gross_adj', 'foreign_unadj', 'genre', 'mpaa_rating',
       'number_of_theaters', 'opening_weekend_adj', 'producers',
       'production_budget_adj', 'release_date', 'runtime',
       'worldwide_gross_unadj', 'writers'],
      dtype='object')

In [8]:
# make sure all the data was imported
df.count()

actors                   9965
close_date               9965
composers                9965
days_in_theater          9965
directors                9965
domestic_gross_adj       9965
foreign_unadj            9965
genre                    9965
mpaa_rating              9965
number_of_theaters       9965
opening_weekend_adj      9965
producers                9965
production_budget_adj    9965
release_date             9965
runtime                  9965
worldwide_gross_unadj    9965
writers                  9965
dtype: int64

In [9]:
# taking a peek into the data
df.head().T

,The Other Side of the Mountain Part II,Universal Soldier,Newtown,Mommie Dearest,G.B.F.
actors,[],"[Jean-Claude Van Damme, Dolph Lundgren]",[],[],"[Natasha Lyonne, Megan Mullally]"
close_date,null,null,null,null,null
composers,[],[Tim Simonec],[],[],[]
days_in_theater,null,null,14 days / 2 weeks,null,null
directors,[],[Roland Emmerich],[],[],[]
domestic_gross_adj,"$42,753,500","$75,661,200","$13,900","$59,219,100",null
foreign_unadj,null,null,null,null,null
genre,Romance,Sci-Fi Action,Documentary,Drama,Comedy
mpaa_rating,PG,R,Unrated,PG,R
number_of_theaters,null,"1,925 theaters",3 theaters,null,null


In [10]:
# looks like I need to manually convert a lot of these into their respective format
df.dtypes

actors                   object
close_date               object
composers                object
days_in_theater          object
directors                object
domestic_gross_adj       object
foreign_unadj            object
genre                    object
mpaa_rating              object
number_of_theaters       object
opening_weekend_adj      object
producers                object
production_budget_adj    object
release_date             object
runtime                  object
worldwide_gross_unadj    object
writers                  object
dtype: object

In [11]:
# looks like I have a lot of duplicated indexes, so will just reset index to handle
    df[df.index.duplicated()].T

,The Karate Kid,Kicking and Screaming,After Midnight,Together,Three of Hearts,When a Stranger Calls,Legend,The Animation Show,Freaky Friday,Happily Ever After,Renegades,Hero,Running Scared
actors,[],[Parker Posey],[],[],[],"[Camilla Belle, Katie Cassidy, Clark Gregg]","[Emily Browning, Christopher Eccleston, Taron ...",[],"[Lindsay Lohan, Jamie Lee Curtis]",[],"[Kiefer Sutherland, Lou Diamond Phillips]","[Donnie Yen, Jet Li, Zhang Ziyi]","[Paul Walker, Vera Farmiga]"
close_date,null,"April 11, 1996","August 7, 2005","October 9, 2003","December 8, 2005","March 19, 2006","January 7, 2016",null,"January 15, 2004",null,null,"November 25, 2004","April 2, 2006"
composers,[],[],[],[],[],[],[Carter Burwell],[],[Rolfe Kent],[],[Michael Kamen],[Tan Dun],[Mark Isham]
days_in_theater,null,null,119 days / 17 weeks,null,51 days / 7.3 weeks,49 days / 7 weeks,49 days / 7 weeks,59 days / 8.4 weeks,163 days / 23.3 weeks,null,null,null,42 days / 6 weeks
directors,[],[Noah Baumbach],[],[],[],[Simon West],[Brian Helgeland],[],[Mark S. Waters],[],[],[Zhang Yimou],[Wayne Kramer]
domestic_gross_adj,"$233,796,000","$1,428,700","$68,900","$1,652,500","$28,900","$63,204,700","$1,862,500","$332,700","$158,113,900","$6,893,600","$19,642,600","$74,813,500","$9,053,000"
foreign_unadj,null,null,"$1,525,534","$13,535,226",null,"$19,106,773","$41,100,000",null,"$50,616,000",null,null,"$123,684,413","$2,524,892"
genre,Sports Action,Comedy,Foreign,Foreign,Documentary,Horror Thriller,Crime Drama,Animation,Fantasy Comedy,Animation,Action Comedy,Foreign / Action,Action / Crime
mpaa_rating,PG,R,Unrated,PG,Unrated,PG-13,R,Unrated,PG,G,R,PG-13,R
number_of_theaters,"1,111 theaters",26 theaters,3 theaters,47 theaters,1 theaters,"3,004 theaters",107 theaters,6 theaters,"3,067 theaters","1,018 theaters","1,627 theaters","2,175 theaters","1,611 theaters"


# Cleaning the data

In [12]:
# reset the index
df = df.reset_index()

# Replaces all unknown values
df.replace('null', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df['genre'].replace('Unknown', np.nan, inplace=True)
df['mpaa_rating'].replace('Unknown', np.nan, inplace=True)
df['release_date'].replace('TBD', np.nan, inplace=True)
df['foreign_unadj'].replace('n/a', np.nan, inplace=True)

# remove things like '2018'
df['release_date'].replace(re.compile(r'^\d+$'), np.nan, inplace=True)

# remove things like 'July 1994'
df['release_date'].replace(re.compile(r'^^\w+\s+\d+$'), np.nan, inplace=True)

# removes encoded characters 
# .replace('\\xa0', '', inplace=True) didn't work as well as passing in the argument re.compile(r'^\\xa0')
df['close_date'] = df['close_date'].str.strip()

In [13]:
# Removes all empty arrays.
# **This will break if done twice!
df['actors'] = df['actors'].apply(lambda x: np.nan if len(x)==0 else x)
df['composers'] = df['composers'].apply(lambda x: np.nan if len(x)==0 else x)
df['directors'] = df['directors'].apply(lambda x: np.nan if len(x)==0 else x)
df['producers'] = df['producers'].apply(lambda x: np.nan if len(x)==0 else x)
df['writers'] = df['writers'].apply(lambda x: np.nan if len(x)==0 else x)

In [14]:
# Checking encoded characters are removed
mask = df['close_date'] == '\xa0December 13, 2007'
df[mask]

,index,actors,close_date,composers,days_in_theater,directors,domestic_gross_adj,foreign_unadj,genre,mpaa_rating,number_of_theaters,opening_weekend_adj,producers,production_budget_adj,release_date,runtime,worldwide_gross_unadj,writers


# Transformation

In [15]:
# changing the release date to a datetime
df['release_date'] = pd.to_datetime(df['release_date'], format="%B %d, %Y");
df['close_date'] = pd.to_datetime(df['close_date'], format="%B %d, %Y");

In [16]:
# remove everything after days and the whitespace before it
df['days_in_theater'] = df['days_in_theater'].replace(re.compile('\sdays.*'), '')

# kill the commas
df['days_in_theater'] = df['days_in_theater'].replace(re.compile(','), '')

# strip any whitespace
df['days_in_theater'] = df['days_in_theater'].str.strip()

# convert values to ints
# df['days_in_theater'] = df[df['days_in_theater'].notnull()]['days_in_theater'].apply(lambda days: int(days))
df['days_in_theater'] = pd.to_numeric(df['days_in_theater'], errors='coerce', downcast='integer')

In [17]:
# remove the dollar signs
df['domestic_gross_adj'] = df['domestic_gross_adj'].replace(re.compile('\$'), '')

# remove the commas
df['domestic_gross_adj'] = df['domestic_gross_adj'].replace(re.compile(','), '')

# kill any encoded whitespace, or whitespace
df['domestic_gross_adj'] = df['domestic_gross_adj'].str.strip()

# convert to int
# df['domestic_gross_adj'] = df[df['domestic_gross_adj'].notnull()]['domestic_gross_adj'].apply(lambda days: int(days))
df['domestic_gross_adj'] = pd.to_numeric(df['domestic_gross_adj'], errors='coerce')

In [18]:
# remove the dollar signs
df['foreign_unadj'] = df['foreign_unadj'].replace(re.compile('\$'), '')

# remove the commas
df['foreign_unadj'] = df['foreign_unadj'].replace(re.compile(','), '')

# kill any encoded whitespace, or whitespace
df['foreign_unadj'] = df['foreign_unadj'].str.strip()

# convert to int
# df['foreign_unadj'] = df[df['foreign_unadj'].notnull()]['foreign_unadj'].apply(lambda days: int(days))
df['foreign_unadj'] = pd.to_numeric(df['foreign_unadj'], errors='coerce')

In [19]:
df['number_of_theaters'] = df['number_of_theaters'].replace(re.compile(r'\stheaters'), '')
df['number_of_theaters'] = df['number_of_theaters'].replace(re.compile(','), '')
df['number_of_theaters'] = df['number_of_theaters'].str.strip()
# df['number_of_theaters'] = df[df['number_of_theaters'].notnull()]['number_of_theaters'].apply(lambda days: int(days)) 
df['number_of_theaters'] = pd.to_numeric(df['number_of_theaters'], errors='coerce')

In [20]:
df['opening_weekend_adj'] = df['opening_weekend_adj'].replace(re.compile('\$'), '')
df['opening_weekend_adj'] = df['opening_weekend_adj'].replace(re.compile(','), '')
df['opening_weekend_adj'] = df['opening_weekend_adj'].str.strip()
df['opening_weekend_adj'] = pd.to_numeric(df['opening_weekend_adj'], errors='coerce')

In [21]:
# # prototyping the millionOrThousand function

# string = '1223'
# million = re.compile(r'(\d+)\s(million)')
# thousand = re.compile(r'(^\d+$)')

# millions = re.findall(million, string)
# thousands = re.findall(thousand, string)

# if millions:
#     print('million')
# if thousands:
#     print('thousand')
    
# thousands[0]

In [22]:
def millionOrThousand(string):
    '''Returns a integer 
    '''
    strng = str(string)
    million = re.compile(r'(\d+)\s(million)')
    thousand = re.compile(r'(^\d+$)')
    
    thousands = re.findall(thousand, strng)
    millions = re.findall(million, strng)
    
    if millions:
        return int(float(millions[0][0]) * 1000000)
    elif thousands:
        return int(thousands[0])

# # production budget -> integer
df['production_budget_adj'] = df['production_budget_adj'].replace(re.compile('\$'), '')
df['production_budget_adj'] = df['production_budget_adj'].replace(re.compile(','), '')
df['production_budget_adj'] = df['production_budget_adj'].str.strip()
df['production_budget_adj'] = (df['production_budget_adj'].dropna().apply(millionOrThousand).astype(np.int))

In [23]:
def hoursAndMins(string):
    string = str(string)
    r_hrsmins = re.compile(r'(^\d+)\shrs\.\s(\d+)')
    hrsmins = r_hrsmins.findall(string)

    if hrsmins:
        hrs = int(hrsmins[0][0]) * 60
        mins = int(hrsmins[0][1])
        return int(hrs + mins)
    else:
        return np.nan
    
df['runtime'] = df['runtime'].apply(hoursAndMins)

In [24]:
# worldwide gross -> integer

# remove the dollar signs
df['worldwide_gross_unadj'] = df['worldwide_gross_unadj'].replace(re.compile('\$'), '')

# remove the commas
df['worldwide_gross_unadj'] = df['worldwide_gross_unadj'].replace(re.compile(','), '')

# kill any encoded whitespace, or whitespace
df['worldwide_gross_unadj'] = df['worldwide_gross_unadj'].str.strip()

# convert to int
# df['domestic_gross_adj'] = df[df['domestic_gross_adj'].notnull()]['domestic_gross_adj'].apply(lambda days: int(days))
df['worldwide_gross_unadj'] = pd.to_numeric(df['worldwide_gross_unadj'], errors='coerce')

# Confirming the data looks good

In [25]:
# check columns for any problems
df.columns

Index(['index', 'actors', 'close_date', 'composers', 'days_in_theater',
       'directors', 'domestic_gross_adj', 'foreign_unadj', 'genre',
       'mpaa_rating', 'number_of_theaters', 'opening_weekend_adj', 'producers',
       'production_budget_adj', 'release_date', 'runtime',
       'worldwide_gross_unadj', 'writers'],
      dtype='object')

In [37]:
df.dtypes

index                            object
actors                           object
close_date               datetime64[ns]
composers                        object
days_in_theater                 float64
directors                        object
domestic_gross_adj              float64
foreign_unadj                   float64
genre                            object
mpaa_rating                      object
number_of_theaters              float64
opening_weekend_adj             float64
producers                        object
production_budget_adj           float64
release_date             datetime64[ns]
runtime                         float64
worldwide_gross_unadj           float64
writers                          object
dtype: object

In [38]:
df.describe()

,days_in_theater,domestic_gross_adj,foreign_unadj,number_of_theaters,opening_weekend_adj,production_budget_adj,runtime,worldwide_gross_unadj
count,5745.00,9470.00,4047.00,9086.00,8417.00,2419.00,9557.00,5080.00
mean,85.60,36773478.15,60885921.46,958.04,9238846.09,48340876.05,104.52,86222892.04
std,103.56,77886268.39,119104852.31,1217.06,19096666.27,47608964.42,22.05,170096362.46
min,-168.00,100.00,29.00,1.00,100.00,220.00,37.00,29.00
25%,28.00,115525.00,2481470.50,9.00,16000.00,15000000.00,91.00,1231547.25
50%,70.00,2838650.00,16002944.00,129.00,228700.00,32000000.00,100.00,19594619.50
75%,119.00,40450025.00,64375453.00,1862.75,11485800.00,68000000.00,113.00,96617784.00
max,3934.00,1234649200.00,2027457462.00,4468.00,246541600.00,300000000.00,583.00,2787965087.00


In [44]:
df[['close_date', 'release_date', 'genre', 'mpaa_rating']].describe()

,close_date,release_date,genre,mpaa_rating
count,5982,9829,9308,9927
unique,1691,2778,65,8
top,2011-09-29 00:00:00,2013-09-06 00:00:00,Foreign,R
freq,20,19,1619,3217
first,1982-03-18 00:00:00,1921-03-06 00:00:00,NaN,NaN
last,2017-04-06 00:00:00,2023-06-30 00:00:00,NaN,NaN


In [32]:
# the original star wars film
# http://www.boxofficemojo.com/movies/?id=starwars4.htm&adjust_yr=2017&p=.htm
df[df['domestic_gross_adj'] == 1234649200.00]

,index,actors,close_date,composers,days_in_theater,directors,domestic_gross_adj,foreign_unadj,genre,mpaa_rating,number_of_theaters,opening_weekend_adj,producers,production_budget_adj,release_date,runtime,worldwide_gross_unadj,writers
8577,Star Wars,"[Kenny Baker, Anthony Daniels, Alec Guinness, ...",NaT,[John Williams],nan,[George Lucas],1234649200.00,314400000.00,Sci-Fi Fantasy,PG,1750.00,nan,[George Lucas],11000000.00,1977-05-25,121.00,775398007.00,[George Lucas]


In [33]:
# lol... twilight
df[df['number_of_theaters'] == 4468.00]

,index,actors,close_date,composers,days_in_theater,directors,domestic_gross_adj,foreign_unadj,genre,mpaa_rating,number_of_theaters,opening_weekend_adj,producers,production_budget_adj,release_date,runtime,worldwide_gross_unadj,writers
3177,The Twilight Saga: Eclipse,"[Kristen Stewart, Robert Pattinson, Taylor Lau...",2010-10-21,[Howard Shore],114.00,[David Slade],326993600.00,397959596.00,Romance,PG-13,4468.00,72736500.00,"[Marty Bowen, Wyck Godfrey, Karen Rosenfelt]",68000000.00,2010-06-30,124.00,698491347.00,[Melissa Rosenberg]


In [34]:
# the star wars franchise seems incredibly successful
df[df['opening_weekend_adj'] == 246541600.00]

,index,actors,close_date,composers,days_in_theater,directors,domestic_gross_adj,foreign_unadj,genre,mpaa_rating,number_of_theaters,opening_weekend_adj,producers,production_budget_adj,release_date,runtime,worldwide_gross_unadj,writers
3438,Star Wars: The Force Awakens,"[John Boyega, Daisy Ridley, Adam Driver, Oscar...",2016-06-02,[John Williams],168.00,[J.J. Abrams],935195600.00,1131561399.00,Sci-Fi Fantasy,PG-13,4134.00,246541600.00,"[J.J. Abrams, Bryan Burk, Kathleen Kennedy]",245000000.00,2015-12-18,136.00,2068223624.00,"[J.J. Abrams, Michael Arndt, Lawrence Kasdan]"


In [26]:
# make sure all the data was imported
df.count()

index                    9965
actors                   4648
close_date               5982
composers                2552
days_in_theater          5745
directors                3491
domestic_gross_adj       9470
foreign_unadj            4047
genre                    9308
mpaa_rating              9927
number_of_theaters       9086
opening_weekend_adj      8417
producers                2926
production_budget_adj    2419
release_date             9829
runtime                  9557
worldwide_gross_unadj    5080
writers                  2375
dtype: int64

In [27]:
df['runtime'].describe()

count   9557.00
mean     104.52
std       22.05
min       37.00
25%       91.00
50%      100.00
75%      113.00
max      583.00
Name: runtime, dtype: float64

In [29]:
# the 583 is a movie called dekalog, a polish tv series
df[df['runtime'] == 583.00]

,index,actors,close_date,composers,days_in_theater,directors,domestic_gross_adj,foreign_unadj,genre,mpaa_rating,number_of_theaters,opening_weekend_adj,producers,production_budget_adj,release_date,runtime,worldwide_gross_unadj,writers
5107,Dekalog (2016 re-release),NaN,NaT,NaN,35.00,NaN,107800.00,nan,Drama,Unrated,5.00,18800.00,NaN,nan,2016-09-02,583.00,nan,NaN


In [30]:
df['mpaa_rating'].value_counts()

R                3217
Unrated          2826
PG-13            2007
PG               1353
G                 262
Not Yet Rated     239
NC-17              21
M                   2
Name: mpaa_rating, dtype: int64

# Looks good, Pickle it

In [45]:
with open('movies.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)